In [1]:
# Algoritmos a serem utilizados
# k-Nearest Neighbors (kNN), Random Fo-rests, XGBoost, Support Vector Regression (SVR) e Neural Networks.

In [1]:
# Importando os pacotes necessários para a análise

import os

import pandas            as pd
# import geopandas         as gpd
import numpy             as np
import scipy             as sc
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_log_error,make_scorer
# import seaborn           as sns

# from google_drive_downloader import GoogleDriveDownloader as gdd
# from sklearn.preprocessing import OneHotEncoder
# from sklearn.preprocessing import OrdinalEncoder

# import mapclassify

pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.reset_option('max_colwidth')

In [2]:
# Lendo a base de dados
df = pd.read_csv("dataset/preprocessed.csv")
print("Formato dos dados: ", df.shape)
print("#Exemplos: {}".format(df.shape[0]))
print("#Atributos: {}".format(df.shape[1]))

Formato dos dados:  (449633, 12)
#Exemplos: 449633
#Atributos: 12


In [5]:
df.head()

,created_on,lat,lon,surface_total_in_m2,surface_covered_in_m2,rooms,expenses,PH,apartment,house,store,price
0,0.786688,0.311355,0.664401,-0.001542,-0.001534,0.2,0.005237,0.0,1.0,0.0,0.0,900000.0
1,0.808340,0.124457,0.537113,-0.001539,-0.001531,0.2,-0.026410,0.0,1.0,0.0,0.0,750000.0
2,0.455493,0.259170,0.591096,-0.001515,-0.001507,0.2,-0.026410,0.0,1.0,0.0,0.0,880000.0
3,0.605453,0.312925,0.668791,-0.001522,-0.001514,0.2,-0.026410,0.0,0.0,1.0,0.0,580000.0
4,0.612670,0.363826,0.668130,-0.001524,-0.001478,0.4,-0.026410,0.0,0.0,1.0,0.0,420000.0


# k-Nearest Neighbors (kNN)

In [24]:
from sklearn.neighbors import KNeighborsRegressor

In [25]:
train, test = train_test_split(df, test_size=0.25)

x_train = train.drop('price', axis=1)
y_train = train['price']

x_test = test.drop('price', axis=1)
y_test = test['price']

In [23]:
grid_params = {
'n_neighbors': [5,7,9,11,13],
'p': [1,2],

}

scorer = make_scorer(mean_squared_log_error, greater_is_better=False)

knn = Pipeline(['scaler', 
                
                KNeighborsRegressor(algorithm='auto', weights='distance')

gs = GridSearchCV(knn, grid_params, verbose=4, n_jobs=-1, scoring=scorer, return_train_score=True)

In [24]:
gs_results = gs.fit(x_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


In [25]:
gs_results.best_score_

-0.18013264645552116

In [26]:
gs_results.best_estimator_

KNeighborsRegressor(n_neighbors=11, p=1, weights='distance')

In [27]:
gs_results.best_params_

{'n_neighbors': 11, 'p': 1}

In [26]:
model = KNeighborsRegressor(n_neighbors=11,p=1,weights='distance')
y_predicted = model.fit(x_train, y_train).predict(x_test)

In [27]:
np.sqrt(mean_squared_log_error(y_test, y_predicted))

0.3862371701271055

In [28]:
# Calculate the absolute errors
errors = abs(y_predicted - y_test)

# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)# Calculate and display accuracy
accuracy = 100 - np.mean(mape)

accuracy

72.40720144388901

# Random Forest

In [80]:
from sklearn.ensemble import RandomForestRegressor
from pprint import pprint
from sklearn.model_selection import RandomizedSearchCV

In [81]:
train, test = train_test_split(df, test_size=0.25)

x_train = train.drop('price', axis=1)
y_train = train['price']

x_test = test.drop('price', axis=1)
y_test = test['price']

In [7]:
# number of trees
n_estimators = [100, 200, 500]

# max number of features to consider at every split
max_features = ['auto', 'sqrt', 'log2']

# max number of levels in tree
max_depth = [10, 20, 50]
max_depth.append(None)

# minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 15]

# minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 5, 10]

random_grid_params = {
    'n_estimators': n_estimators,
    'max_features': max_features,
    'max_depth': max_depth,
    'min_samples_split': min_samples_split,
    'min_samples_leaf': min_samples_leaf,
}

pprint(random_grid_params)

{'max_depth': [10, 20, 50, None],
 'max_features': ['auto', 'sqrt', 'log2'],
 'min_samples_leaf': [1, 2, 5, 10],
 'min_samples_split': [2, 5, 10, 15],
 'n_estimators': [100, 200, 500]}


In [8]:
# create the base model to tune
rfr = RandomForestRegressor()

# use the random grid to search for best hyperparameters,
# using 5 fold cross validation, earch across 20 different combinations, and use all available cores
rfr_random = RandomizedSearchCV(estimator=rfr, param_distributions=random_grid_params, n_iter=20, cv=3, verbose=2, random_state=42, n_jobs=-1)

In [9]:
# fit the random search model
rfr_random_results = rfr_random.fit(x_train, y_train)

Fitting 3 folds for each of 20 candidates, totalling 60 fits


/home/thay/anaconda3/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


In [10]:
rfr_random_results.best_score_

0.6869087933350606

In [11]:
rfr_random_results.best_params_

{'n_estimators': 100,
 'min_samples_split': 10,
 'min_samples_leaf': 1,
 'max_features': 'log2',
 'max_depth': None}

In [82]:
# # max number of features to consider at every split
max_features = ['auto', 'log2']

# minimum number of samples required at each leaf node
min_samples_split = [2, 5, 10]

grid_params = {
     'max_features': max_features,
    'min_samples_split': min_samples_split,
}

pprint(grid_params)

{'max_features': ['auto', 'log2'], 'min_samples_split': [2, 5, 10]}


In [6]:
scorer = make_scorer(mean_squared_log_error,greater_is_better=False)

rfr = RandomForestRegressor()

gs = GridSearchCV(rfr, grid_params, verbose=4, n_jobs=-1, scoring=scorer, return_train_score=True)

In [ ]:
gs_results = gs.fit(x_train, y_train)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


In [19]:
gs_results.best_score_

-0.08600977146548427

In [20]:
gs_results.best_estimator_

RandomForestRegressor(n_jobs=-1)

In [21]:
gs_results.best_params_

{'min_samples_split': 2}

In [18]:
model = RandomForestRegressor(n_jobs=-1, n_estimators=100, min_samples_split=2, min_samples_leaf=1, max_features='auto', max_depth=None)
y_predicted = model.fit(x_train, y_train).predict(x_test)

In [19]:
np.sqrt(mean_squared_log_error(y_test, y_predicted))

0.2561500180491964

In [20]:
# Calculate the absolute errors
errors = abs(y_predicted - y_test)

# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)# Calculate and display accuracy
accuracy = 100 - np.mean(mape)

accuracy

81.64378991846057

# Support Vector Regression (SVR)

In [39]:
from sklearn.svm import SVR

In [40]:
X = df.drop("price",1)
y = df.price
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [42]:
model = SVR()
y_predicted = model.fit(X_train, y_train).predict(X_test)

In [ ]:
np.sqrt(mean_squared_log_error(y_test,y_predicted))

# XGBoost

In [60]:
from xgboost import XGBRegressor
from sklearn.model_selection import KFold

In [61]:
train, test = train_test_split(df, test_size=0.25)

x_train = train.drop('price', axis=1)
y_train = train['price']

x_test = test.drop('price', axis=1)
y_test = test['price']

In [62]:
xgboost = XGBRegressor(n_jobs=-1)

random_grid_params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
}

xgboost_random = RandomizedSearchCV(estimator=xgboost, param_distributions=random_grid_params, n_iter=30, cv=3, verbose=2, random_state=42, n_jobs=-1)

In [63]:
# fit the random search model
xgboost_random_results = xgboost_random.fit(x_train, y_train)

Fitting 3 folds for each of 30 candidates, totalling 90 fits


In [64]:
xgboost_random_results.best_score_

0.6452017695916566

In [65]:
xgboost_random_results.best_params_

{'subsample': 1.0,
 'min_child_weight': 1,
 'max_depth': 5,
 'gamma': 0.5,
 'colsample_bytree': 0.8}

In [67]:
model = XGBRegressor(n_jobs=-1,subsample=1.0, min_child_weight=1,max_depth=5, gamma=0.5, colsample_bytree=0.8)
y_predicted = model.fit(x_train, y_train).predict(x_test)

In [73]:
y_predicted[y_predicted < 0] = 0

In [78]:
np.sqrt(mean_squared_log_error(y_test, y_predicted))

0.5638860220847428

In [76]:
# Calculate the absolute errors
errors = abs(y_predicted - y_test)

# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)# Calculate and display accuracy
accuracy = 100 - np.mean(mape)

accuracy

65.91542190885194

# Neural Networks
## Will use the same NN as shown in https://www.kaggle.com/luckeciano/nn-with-tabular-data-baseline

In [35]:
#Import Keras 
from keras.layers import Input, Dense, BatchNormalization
from keras.models import Model, Sequential
from keras.layers import LeakyReLU
from keras.optimizers import Adam
from keras import backend as K
from keras.utils.vis_utils import plot_model
from keras.callbacks import LearningRateScheduler, TensorBoard
from sklearn.metrics import mean_squared_error
from keras.losses import MeanSquaredLogarithmicError
from keras.metrics import mean_squared_logarithmic_error

In [43]:
#Hyperparameters
ACTIVATION = 'relu'
EPOCHS = 2
BATCH_SIZE = 128
LEARNING_RATE= 0.01
LOSS_FUNCTION = 'mean_squared_error'
BETA_1 = 0.9
BETA_2 = 0.999
EPSILON = 1e-08
EPOCHS_DROP = 20
DROP = 0.8

In [13]:
X = df.drop("price",1)
y = df.price
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [18]:
#Build Neural Network
model =  Sequential()
model.add(Dense(128,  input_dim=10,  activation = ACTIVATION))
model.add(Dense(128, activation = ACTIVATION))
model.add(Dense(64,  activation = ACTIVATION))
model.add(Dense(1))

In [27]:
#Define loss metric
def root_mean_squared_error(y_true, y_pred):
    return np.sqrt(mean_squared_log_error(y_true,y_pred))

In [44]:
# adam = Adam(lr=LEARNING_RATE, beta_1=BETA_1, beta_2=BETA_2, epsilon=EPSILON)
# model.compile (loss = MeanSquaredLogarithmicError(), optimizer = adam)
# model.fit (X_train, y_train, epochs = EPOCHS, batch_size = BATCH_SIZE, verbose = 1, validation_split=0.1)

Epoch 1/2
2119/2119 [==============================] - 4s 2ms/step - loss: 0.3581 - val_loss: 0.3522
Epoch 2/2
2119/2119 [==============================] - 4s 2ms/step - loss: 0.3582 - val_loss: 0.3508


In [46]:
model = Sequential()

# The Input Layer :
model.add(Dense(128, kernel_initializer='normal',input_dim = 10, activation='relu'))

# The Hidden Layers :
model.add(Dense(256, kernel_initializer='normal',activation='relu'))
model.add(Dense(256, kernel_initializer='normal',activation='relu'))
model.add(Dense(256, kernel_initializer='normal',activation='relu'))

# The Output Layer :
model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
model.compile(loss=MeanSquaredLogarithmicError(), optimizer='adam', metrics=['mean_squared_error'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 128)               1408      
_________________________________________________________________
dense_13 (Dense)             (None, 256)               33024     
_________________________________________________________________
dense_14 (Dense)             (None, 256)               65792     
_________________________________________________________________
dense_15 (Dense)             (None, 256)               65792     
_________________________________________________________________
dense_16 (Dense)             (None, 1)                 257       
Total params: 166,273
Trainable params: 166,273
Non-trainable params: 0
_________________________________________________________________


In [50]:
model.fit(X_train, y_train, epochs=15, batch_size=32, validation_split = 0.2)

Epoch 1/15
7532/7532 [==============================] - 20s 3ms/step - loss: 0.3790 - mean_squared_error: 2262695673856.0000 - val_loss: 0.3839 - val_mean_squared_error: 47172785453137920.0000
Epoch 2/15
7532/7532 [==============================] - 23s 3ms/step - loss: 0.3777 - mean_squared_error: 2252686753792.0000 - val_loss: 0.3800 - val_mean_squared_error: 55737577306587136.0000
Epoch 3/15
7532/7532 [==============================] - 24s 3ms/step - loss: 0.3759 - mean_squared_error: 2243038019584.0000 - val_loss: 0.3746 - val_mean_squared_error: 64208618613899264.0000
Epoch 4/15
7532/7532 [==============================] - 24s 3ms/step - loss: 0.3747 - mean_squared_error: 2234653081600.0000 - val_loss: 0.3761 - val_mean_squared_error: 72057903275573248.0000
Epoch 5/15
7532/7532 [==============================] - 24s 3ms/step - loss: 0.3730 - mean_squared_error: 2227799588864.0000 - val_loss: 0.3739 - val_mean_squared_error: 71138535461093376.0000
Epoch 6/15
7532/7532 [=============

In [51]:
y_predicted = model.predict(X_test)
np.sqrt(mean_squared_log_error(y_test,y_predicted))

0.5995757353337279